# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator as data_augment
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Data Importation

In [2]:
#Uncomment below to clone the dataset from github

!git clone https://github.com/echefulouis/Gender-Classification-Model.git


Cloning into 'Gender-Classification-Model'...
remote: Enumerating objects: 58653, done.
remote: Total 58653 (delta 0), reused 0 (delta 0), pack-reused 58653
Receiving objects: 100% (58653/58653), 261.43 MiB | 15.82 MiB/s, done.
Updating files: 100% (58659/58659), done.


#Data Preprocessing and Augmentation

In [3]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory('/content/Gender-Classification-Model/GC dataset/Training', target_size=(224, 224), batch_size=32)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_datagen.flow_from_directory('/content/Gender-Classification-Model/GC dataset/Validation', target_size=(224, 224), batch_size=32)


Found 47009 images belonging to 2 classes.
Found 11649 images belonging to 2 classes.


# Load the pre-trained MobileNetV2 model

In [4]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


9406464/9406464 [==============================] - 2s 0us/step


In [5]:
# Add a custom classification layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [6]:
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Model Training 

In [7]:
# Train the model
history = model.fit(train_generator, steps_per_epoch=train_generator.samples//train_generator.batch_size, epochs=10, validation_data=val_generator, validation_steps=val_generator.samples//val_generator.batch_size)


Epoch 1/10
1469/1469 [==============================] - 248s 145ms/step - loss: 0.1391 - accuracy: 0.9510 - val_loss: 2.1099 - val_accuracy: 0.5883
Epoch 2/10
1469/1469 [==============================] - 213s 145ms/step - loss: 0.1016 - accuracy: 0.9648 - val_loss: 0.6944 - val_accuracy: 0.8706
Epoch 3/10
1469/1469 [==============================] - 214s 145ms/step - loss: 0.0902 - accuracy: 0.9683 - val_loss: 1.0179 - val_accuracy: 0.8040
Epoch 4/10
1469/1469 [==============================] - 212s 144ms/step - loss: 0.0833 - accuracy: 0.9704 - val_loss: 3.7560 - val_accuracy: 0.5901
Epoch 5/10
1469/1469 [==============================] - 233s 158ms/step - loss: 0.0777 - accuracy: 0.9717 - val_loss: 0.1990 - val_accuracy: 0.9560
Epoch 6/10
1469/1469 [==============================] - 213s 145ms/step - loss: 0.0704 - accuracy: 0.9751 - val_loss: 0.1687 - val_accuracy: 0.9649
Epoch 7/10
1469/1469 [==============================] - 214s 145ms/step - loss: 0.0662 - accuracy: 0.9759 - val_

# Model Evaluation

In [8]:
# Evaluate the model
model.evaluate(val_generator, steps=val_generator.samples//val_generator.batch_size)

364/364 [==============================] - 21s 58ms/step - loss: 0.1704 - accuracy: 0.9605


[0.17036589980125427, 0.9605082273483276]

In [9]:
# Evaluate the model on the validation dataset
validation_accuracy = model.evaluate(val_generator, steps=val_generator.samples//val_generator.batch_size)[1]

print("Validation accuracy: {:.2f}%".format(validation_accuracy * 100))

364/364 [==============================] - 20s 56ms/step - loss: 0.1704 - accuracy: 0.9605
Validation accuracy: 96.05%
